# Assignment 2


In this assignment we will cover topics from the previous 3 lectures. We will cover the following topics:

1) Training a simple Linear Model

2) Implementing Modules with Backprop functionality

3) Implementing Convolution Module on Numpy.

4) Implement Dropout/Different Optimizer setups.

5) Implementing Pool and Training on CIFAR10?


It is crucial to get down to the nitty gritty of the code to implement all of these. No external packages (like caffe,pytorch etc), which directly give functions for these steps, are to be used. 

# Training a simple Linear Model

In this section, you will write the code to train a Linear Model. The goal is to classify and input $x_n$ of size $n$ into one of $m$ classes. For this goal, you need to create the following parts:

1) ** A weight Matrix $W_{n\times m}$ **, where the Weights are multipled to the input $X_n$ (Vector of size $n$), to find $m$ scores $S_m$ for the $m$ classes.

2) ** The Loss function **: We learnt two Kinds of Loss functions:
  *  The Hinge Loss: This loss measures, for each sample, how many times were the wrong classes scored above correct class score - $\Delta$ ? and by how much? This leads to the formulation:
  
$$
L_i = \sum_{j\neq y_i} \max(0, s_j - s_{y_i} + \Delta)
$$

where $y_i$ is the correct class, and $s_j$ is the score for the $j$-th class (the $j$-th element of $S_m$)
  
  * The softmax Loss: By interpreting the scores as unnormalized log probabilities for each class, this loss tries to measure dissatisfaction with the scores in terms of the log probability of the right class:

$$
L_i = -\log\left(\frac{e^{f_{y_i}}}{ \sum_j e^{f_j} }\right) \hspace{0.5in} \text{or equivalently} \hspace{0.5in} L_i = -f_{y_i} + \log\sum_j e^{f_j}
$$

where $f_{ y_i }$ is the $i$-th element of the output of $W^T_{n \times m} . X_m$

4) ** Regularization term **: In addition to the loss, you need a Regularization term to lead to a more distributed( in case of $L_2$) or sparse (in case of $L_1$) learning of the weights. For example, and having $L_2$ regularization would imply that your loss has the following additional term:

$$
R(W) = \sum_k\sum_l W_{k,l}^2,
$$

making the total loss:
$$
L =  \underbrace{ \frac{1}{N} \sum_i L_i }_\text{data loss} + \underbrace{ \lambda R(W) }_\text{regularization loss} \\\\
$$

3) ** An Optimization Procedure **: This refers to the process which tweaks the weight Matrix $W_{n\times m}$ to reduce the loss function $L$. In our case, this refers to Mini-batch Gradient Descent algorithm. We adjust the weights $W_{n\times m}$, based on the gradient of the loss $L$ w.r.t. $W_{n\times m}$. This leads to:

$$
W_{t+1} = W_{t} + \alpha \frac{\partial L}{\partial W},
$$
where $\alpha$ is the learning rate. Additionally, as we will be doing "mini-batch" gradient Descent, instead of finding loss over the whole dataset, we find it only for a small sample of the traning data for each learning step we take. Basically,
$$
W_{t+1} = W_{t} + \alpha \frac{\partial \sum^{b}{L_{x_i}}}{\partial W},
$$
where, $b$ is the batch size.

# Question 1

Train a **Single-Layer Classifier** for the MNIST dataset. Guidelines:
* Use a loss of your choice.
* Keep a validation split of the trainingset for finding the right value of $\lambda$ for the regularization, and to check for over fitting.
* Finally,evaluate the classification performance on the testset.


In [15]:
## Load The Mnist data:
# Download data from http://yann.lecun.com/exdb/mnist/
# load the data.
import numpy as np
import struct
import math

def read_labels(file_name):
    label_file = open(file_name, 'rb')
    magic_nbr, size = struct.unpack(">II", label_file.read(8))
    labels = np.zeros(size)
    for i in range(0, size):
        labels[i] = struct.unpack(">B", label_file.read(1))[0]
    label_file.close()
    return labels

def read_images(file_name):
    image_file = open(file_name, 'rb')
    magic_nbr, size, rows, cols = struct.unpack(">IIII", image_file.read(16))
    images = np.zeros(size*rows*cols)
    for i in range(0, size*rows*cols):
        images[i] = struct.unpack(">B", image_file.read(1))[0]
    image_file.close()
    images = images.reshape(size, rows*cols)
    return images

# split the data into train, and valid
labels = read_labels("train-labels.idx1-ubyte")
images = read_images("train-images.idx3-ubyte")
print labels.shape
print images.shape
train_data = np.c_[images, labels]

shuffled = np.copy(train_data)
np.random.shuffle(shuffled)
ratio = 0.8
l = shuffled.shape[0]
b_pt = int(math.floor(ratio*l))
validation = train_data[0:b_pt,:]
train_data = train_data[b_pt:,:]

train_X = train_data[:,0:-1]
train_Y = train_data[:, -1]

# Now a function, which returns a generator random mini-batch of the input data

def get_minibatch_function(training_x, training_y):
    
    def get_minibatch(training_x=training_x, training_y=training_y):
        ## Read generator functions if required.
        training_x, training_y = np.random.shuffled(training_x, training_y)
        i = 0
        l = training_x.shape[0]
        start  = 0
        while (True):
            end = np.random.randint(low = 50, high = 100)
            end = max(l, start+end)
            mini_x = training_x[start:end,:]
            mini_y = training_y[start:end,:]
            start = end
        ## WRITE CODE HERE
        
        yield mini_x,mini_y
        
    return get_minibatch

(60000,)
(60000, 784)


In [ ]:
# Define the class Single Layer Classifier
class single_layer_classifier():
    
    def __init__(self, input_size, output_size):
        
        ## WRITE CODE HERE
        self.weights = np.random.rand((input_size, output_size))
        self.lambada = 1000
        # Give the instance a weight matrix, initialized randomly.
        
    # Define the forward function
    def forward(self, input_x):
        
        # get the scores
        
        ## WRITE CODE HERE
        scores =  np.matmul(self.weights, input_x)
        return scores
    
    # Similarly a backward function
    # we define 2 backward functions (as Loss = L1 + L2, grad(Loss) = grad(L1) + grad(L2))
    
    def backward_from_loss(self, grad_from_loss):
        
        # this function returns a matrix of the same size as the weights, 
        # where each element is the partial derivative of the loss w.r.t. the respective element of weight.
        
        ## WRITE CODE HERE
        
        return grad_matrix
        
    def backward_from_l2(self):
        
        # this function returns a matrix of the same size as the weights, 
        # where each element is the partial derivative of the regularization_term
        # w.r.t. the respective element of weight.
        
        ## WRITE CODE HERE
        grad_matrix = self.lambda*self.weights
        return grad_matrix
    
    # BONUS
    def grad_checker(input_x, grad_matrix):
        
        # Guess what to do?
        
        ## WRITE CODE HERE
        
        if diff<threshold:
            return true
        else:
            return false

In [ ]:
# Now we need the loss functions,one which calculates the loss, 
# and one which give the backward gradient
# Make any one of the suggested losses

def loss_forward(input_y,scores):

    ## WRITE CODE HERE
    class_scores = np.zeros(len(input_y))
    for i in range(0, len(input_y)):
        class_scores[i] = scores[i, input_y[i]]
        
    common_loss = np.log(np.sum(np.exp(scores), axis = 1))
    loss= common_loss - class_scores
    return loss

def loss_backward(loss):
    # This part deals with the gradient from the loss to the weight matrix.
    # for example, in case of softmax loss(-log(qc)), this part gives grad(loss) w.r.t. qc

    ## WRITE CODE HERE    

    return grad_from_loss
        

In [ ]:
# Finally the trainer:

# let it be for t iterations:

# make an instance of single_layer_classifier,
# get the mini-batch yielder.

for iter,input_x, input_y in enumerate(get_minibatch()):
    
    ## Write code here for each iteration of training.

In [ ]:
# Find the performance on the validation set.
# find the top-1 accuracy on the validation set.

In [ ]:
# now make a trainer function based on the above code, which trains for 't' iteration,
# and returns the performance on the validation

def trainer(iterations, kwargs):

    ## WRITE CODE HERE
    
    return top_1


In [ ]:
# Find the optimal lambda and iterations t

In [ ]:
# Train on whole dataset with these values,(from scratch)
# report final performance on mnist test set.

# Find the best performing class and the worst performing class.


# Implementing Backprop

Now that you have had some experience with single layer networks, its time to go to more complex architectures. But first we need to completely understand and implement backpropagation.

## Backpropagation:

Simple put, a way of computing gradients of expressions through recursive application of chain rule. If,
$$
L = f (g (h (\textbf{x})))
$$
then,
$$
\frac{\partial L}{\partial \textbf{x}} = \frac{\partial f}{\partial g} \times \frac{\partial g}{\partial h} \times\frac{\partial h}{\partial \textbf{x}} 
$$

** Look into the class Lecture for more detail **



# Question 2 : Scalar Backpropagation

Evaluate the gradient of the following functions w.r.t. the input.

1) $$ f(x,y,z) =  log(\sigma(\frac{cos(\pi \times \sigma(x))+sin(\pi \times \sigma(y/2))}{z^2}))$$
where $\sigma$ is the sigmoid function. Find gradient for the following values:
  * $(x,y,z)$ =  (1,2,3)
  * $(x,y,z)$ =  (3,2,1)
  * $(x,y,z)$ =  (12,23,31)
  * $(x,y,z)$ =  (32,21,13)

2) $$ f(x,y,z) = -tan(z) + exp(4x^2 + 3x + 10) - x^{y^z} $$
where $\exp$ is the exponential function. Find gradient for the following values:
  * $(x,y,z)$ =  (-0.1 ,2 ,-3)
  * $(x,y,z)$ =  (-3, 0.2,0.5)
  * $(x,y,z)$ =  (1.2, -2.3, 3.1)
  * $(x,y,z)$ =  (3.2, 2.1, -1.3)
      

In [93]:
# To solve this problem, construct the computational graph (will help understanding the problem)(not part of assignment)
# Write each component of the graph as a class, with forward and backward functions.
import math
# for eg:
class sigmoid():
    def __init__(self):
        return 
    
    def forward(self, in1):
        self.x = in1
        return 1.0/(1.0 + math.exp(-1.0*in1))
        # save values useful for backpropagation
        
    def backward(self, dz):
        in1 = self.x
        return self.forward(in1)*(1.0 - self.forward(in1)) * dz

class cos():
    def __init__(self):
        return
    
    def forward(self, in1):
        self.x = in1
        return math.cos(in1)
    
    def backward(self, back):
        in1 = self.x
        return -1.0 * math.sin(in1) * back

class sin():
    def __init__(self):
        return
    
    def forward(self, in1):
        self.x= in1
        return math.sin(in1)
    
    def backward(self, back):
        in1 = self.x
        return math.cos(in1) * back 

class plus():
    def __init__(self):
        pass
    
    def forward(self, in1, in2):
        return in1 + in2
        
    def backward(self, back):
        return back
    
class mult():
    def __init__(self):
        pass
    
    def forward(self, in1, in2):
        self.x = in1
        self.y = in2
        return in1*in2
    
    def backward(self, back):
        in1 = self.x
        in2 = self.y
        return [in2*back, in1*back]
    
class log():
    def __init__(self):
        return
    
    def forward(self, in1):
        self.x = in1
        out = math.log(in1)
        self.out = out
        return out
    
    def backward(self, back):
        in1 =  self.x
        return (1.0/in1)*back
        
class square():
    def __init__(self):
        return
    
    def forward(self, in1):
        self.x = in1
        return (in1**2)
    
    def backward(self, back):
        in1 =  self.x
        return 2*in1*back
    
class inv():
    def __init__(self):
        return
    
    def forward(self, in1):
        self.x = in1
        return (1.0/in1)
    
    def backward(self, back):
        in1 = self.x
        return (-1.0/(in1**2))*back
    
class const_mult():
    def __init__(self, const):
        self.const = const
    
    def forward(self, in1):
        return in1*self.const
    
    def backward(self, back):
        return back*self.const
    
class by_two():
    def __init__(self):
        pass
    
    def forward(self, in1):
        return in1 * 0.5
    
    def backward(self, back):
        return 0.5 * back

class e_to_the():
    def __int__(self):
        pass
    
    def forward(self, in1):
        self.x = in1
        return math.exp(in1)
    
    def backward(self, back):
        return math.exp(self.x)*back

class tan():
    def __int__(self):
        pass
    
    def forward(self, in1):
        self.x = in1
        return math.tan(in1)
    
    def backward(self, back):
        return (1.0/math.cos(self.x))**2  * back

class x_to_the():
    def __init__(self):
        pass
    
    def forward(self, in1, in2):
        self.x = in1
        self.y = in2
        return in1**in2
    
    def backward(self, back):
        in1 = self.x
        in2 = self.y
        return [in2* (in1**(in2-1)), math.log(in1)*math.exp(in2 * math.log(in1))]

    # CAUTION: Carefully treat the input and output dimension variation. At worst, handle them with if statements.
    # Similarly create the classes for various sub-parts/elements of the graph.

In [94]:
# Now write func_1_creator, 
# which constructs the graph(all operators), forward and backward functions.

class func1():
    def __init__(self):
        # construct the graph here, 
        # assign the instances of function modules to self.var
        self.a1 = sigmoid()
        self.a2 = const_mult(math.pi)
        self.a3 = cos()
        self.b1 = by_two()
        self.b2 = sigmoid()
        self.b3 = const_mult(math.pi) 
        self.b4 = sin()
        self.c1 = square()
        self.c2 = inv()
        self.ab = plus()
        self.abc1 = mult()
        self.abc2 = sigmoid()
        self.abc3 = log()
        
        
    def forward(self,x,y,z):
        # Using the graph element's forward functions, get the output. 
        
        x1 = self.a3.forward(self.a2.forward(self.a1.forward(x)))
        y1 = self.b4.forward(self.b3.forward(self.b2.forward(self.b1.forward(y))))  
        z1 = self.c2.forward(self.c1.forward(z))
        
        xyz = self.abc1.forward(self.ab.forward(x1, y1), z1)
        m = self.abc2.forward(xyz)
        output = self.abc3.forward(m)
        
        #print "x1: {}".format(x1)
        #print "y1: {}".format(y1)
        #print "z1 : {}".format(z1)
        
        return output
    
    def backward(self):
        # Use the saved outputs of each module, and backward() function calls
        grad_x = self.a1.backward(
            self.a2.backward(
            self.a3.backward(
            self.ab.backward(
            self.abc1.backward(
            self.abc2.backward(
            self.abc3.backward(1
            )))[0]))))
        
        grad_y = self.b1.backward(
            self.b2.backward(
            self.b3.backward(
            self.b4.backward(
            self.ab.backward(
            self.abc1.backward(
            self.abc2.backward(
            self.abc3.backward(1
            )))[0])))))
        grad_z = 0
        
        grad_z = self.c1.backward(
            self.c2.backward(
            self.abc1.backward(
            self.abc2.backward(
            self.abc3.backward(1
            )))[1]))
        return [grad_x,grad_y,grad_z]
    
    
# Similarly,
class func2():
    def __init__(self):
        # construct the graph here, 
        # assign the instances of function modules to self.var
        self.a11 = const_mult(3)
        self.a21 = square()
        self.a22 = const_mult(4)
        self.a1a21 = plus()
        self.a1a22 = plus()
        self.a1a23 = e_to_the()
        self.c1 = tan()
        self.c2 = const_mult(-1)
        self.bc = x_to_the()
        self.ac = plus()
        self.abc1 = x_to_the()
        self.abc2 = const_mult(-1)
        self.abc3 = plus()
        
    def forward(self, x,y,z):
        # Using the graph element's forward functions, get the output. 
        p1 = self.a1a23.forward(self.a1a22.forward(self.a1a21.forward(self.a11.forward(x), self.a22.forward(self.a21.forward(x))), 10))
        p2 = self.c2.forward(self.c1.forward(z))
        ac = self.ac.forward(p1, p2)
        output = self.abc3.forward(self.abc2.forward(self.abc1.forward(x, self.bc.forward(y,z))), ac)
        return output
    
    def backward(self):
        # Use the saved outputs of each module, and backward() function calls
        [grad_x,grad_y,grad_z] = [0,0,0]
        common_x = self.a1a21.backward(
            self.a1a22.backward(
            self.a1a23.backward(
            self.ac.backward(
            self.abc3.backward(1
            )))))
        
        sep_x = self.abc1.backward(self.abc2.backward(self.abc3.backward(1)))[0]
        
        grad_x = self.a11.backward(common_x) + self.a21.backward(self.a22.backward(common_x)) + sep_x
        
        grad_y = self.bc.backward(self.abc1.backward(self.abc2.backward(self.abc3.backward(1)))[1])[0]
        
        grad_z1 = self.c1.backward(self.c2.backward(self.ac.backward(self.abc3.backward(1))))
        grad_z2 = self.bc.backward(self.abc1.backward(self.abc2.backward(self.abc3.backward(1)))[1])[1]
        grad_z = grad_z1 + grad_z2
        return [grad_x, grad_y, grad_z]
    
func1 = func1()
values1 = [[1,2,3],[3,2,1],[12,23,31],[32,21,13]]
for i in range(0,4):
    print "funcValue({}) : {}".format(values1[i], func1.forward(values1[i][0],values1[i][1], values1[i][2]))
    print "Grads({}) : {}\n".format(values1[i],func1.backward())

func2 = func2()
values2 = [[0.1 ,2 ,3],[3, 0.2,0.5],[1.2, 2.3, 3.1],[3.2, 2.1, 1.3]]
for i in range(0,4):
    print "funcValue({}) : {}".format(values2[i], func2.forward(values2[i][0],values2[i][1], values2[i][2]))
    print "Grads({}) : {}\n".format(values2[i],func2.backward())


funcValue([1, 2, 3]) : -0.688485604304
Grads([1, 2, 3]) : [-0.025544725443367555, -0.011336065344893414, -0.003100440188579966]

funcValue([3, 2, 1]) : -0.820897330718
Grads([3, 2, 1]) : [-0.011797251236071567, -0.11479644823429057, 0.2699174041853647]

funcValue([12, 23, 31]) : -0.693667590708
Grads([12, 23, 31]) : [-1.9395308189331025e-13, -8.28317899719717e-09, 3.358358006475774e-05]

funcValue([32, 21, 13]) : -0.696109880341
Grads([32, 21, 13]) : [-4.73012415297613e-30, -1.2834237773424494e-07, 0.00045647316954041154]

funcValue([0.1, 2, 3]) : 30946.1725936
Grads([0.1, 2, 3]) : [117594.91417957142, 12, 4.524857927537134]

funcValue([3, 0.2, 0.5]) : 7.69478526514e+23
Grads([3, 0.2, 0.5]) : [2.0775920215883447e+25, 1.118033988749895, -2.0182089259631253]

funcValue([1.2, 2.3, 3.1]) : 255823920.846
Grads([1.2, 2.3, 3.1]) : [3223381665.37192, 17.82338340560419, 10.012492206391777]

funcValue([3.2, 2.1, 1.3]) : 1.99928093249e+26
Grads([3.2, 2.1, 1.3]) : [5.717943466907864e+27, 1.6240864

## Question 3 : Modular Vector Backpropagation

* Construct a Linear Layer module, implementing the forward and backward functions for arbitrary sizes.
* Construct a ReLU module, implementing the forward and backward functions for arbitrary sizes.
* Create a 2 layer MLP using the constructed modules.

* Modifying the functions built in Question 1 , train this two layer MLP for the same data set (MNIST).

In [ ]:
# Class for Linear Layer (Refer code of pytorch/tensorflow package if required.) 


In [ ]:
# Class for ReLU


In [ ]:
# Your 2 layer MLP 


In [ ]:
# Train the MLP


In [ ]:
# Validation Performance


In [ ]:
# Best Class and worst class performance.


# After the lecture on Jan 31st.

# Implementing Convolution Module on Numpy.

* This topic will require you to implement the Convolution operation using Numpy.
* You will implement two methods of doing it, an intuitive and an optimised way.
* Additional operations like dropout, batch norms. 
* We will use the created Module for interesting task like Blurring, Bilateral Filtering.
* Finally, we create the Backprop for this.
* Train a Conv model for the same MNIST dataset. (can be a script based training, instead of having it in jupyter notebook.)
